In [75]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import csv

%matplotlib inline

In [76]:
!ls

Datasets
Exploratory_PCA_dmb.ipynb
FractalDimensionWaveform-KaggleHousing.ipynb
ML_Project_KM.ipynb
README.MD
Wenchang_Qian_Kaggle_Ames_Iowa_Housing_Price_Prediction.ipynb
james_proj3.ipynb
jamesczq_baseline_regressions.ipynb


In [77]:
#whichDataSet = 0;
whichDataSet = 'w';

if whichDataSet == 0: # Using the original data set 
    train = pd.read_csv('Datasets/train.csv')
    test = pd.read_csv('Datasets/test.csv')
    # Concat. train[no ID column, ... (all columns) ..., no SalePrice column]
    #     with test[no ID column, ... (all columns) ..., no SalePrice column]
    # (Test data has no SalePrice column anyway)
    train_test = pd.concat([train.loc[:,'MSSubClass':'SaleCondition'],
                             test.loc[:,'MSSubClass':'SaleCondition']]);
    print('Train-Test mega dataset shape:',train_test.shape)
##
##
if whichDataSet == 'w': # Using Wenchang's modified data set
    train = pd.read_csv('Datasets/train_Wenchang.csv')
    test = pd.read_csv('Datasets/test_Wenchang.csv')
    train.drop(['Unnamed: 0'], axis=1, inplace=True)
    test.drop(['Unnamed: 0'], axis=1, inplace=True)
    train_test = pd.concat([train.loc[:,'1stFlrSF':'RemodYearDiff'],
                             test.loc[:,'1stFlrSF':'RemodYearDiff']]);
    print('Train-Test mega dataset shape:',train_test.shape)

Train-Test mega dataset shape: (2919, 72)


In [78]:
sns.set(style="ticks", color_codes=True)

In [79]:
# scatter plot of various features

# It taxes the computer very much to do the plot.
# Here I only plotted scatter plot between FOUR numeric features
# We can add more features if we want later on

'''
sns.pairplot(train[['LotArea',\
                    'OverallQual',\
                    'OverallCond',\ 
                    'SalePrice']],\
             diag_kind='kde', kind = 'reg',\
             markers='+', diag_kws=dict(shade=True));
sns.set(font_scale = 1.5)
'''

"\nsns.pairplot(train[['LotArea',                    'OverallQual',                    'OverallCond',\\ \n                    'SalePrice']],             diag_kind='kde', kind = 'reg',             markers='+', diag_kws=dict(shade=True));\nsns.set(font_scale = 1.5)\n"

In [80]:
# Zero bedroom or zero garage parking spaces or zero bathroom may give trouble
# to derived features such as bath/bed
# Need to think about a way to walk around this

print('*'*50)
print('For Training Data Set:')
print('*'*50)
print('Num of houses with Zero Bedroom:',\
      sum(train.BedroomAbvGr==0))
print('Num of houses with Zero Garage Parking:',\
      sum(train.GarageCars==0))
print('Num of houses with Zero Full Bath:',\
      sum(train.FullBath==0))
print('Num of houses with Zero Half Bath:',\
      sum(train.HalfBath==0))
print('Num of houses with Neither Half Nor Full Bath:',\
      sum((train.HalfBath==0) & (train.FullBath==0)))
print('*'*50)
print('For Test Data Set:')
print('*'*50)
print('Num of houses with Zero Bedroom:',\
      sum(test.BedroomAbvGr==0))
print('Num of houses with Zero Garage Parking:',\
      sum(test.GarageCars==0))
print('Num of houses with Zero Full Bath:',\
      sum(test.FullBath==0))
print('Num of houses with Zero Half Bath:',\
      sum(test.HalfBath==0))
print('Num of houses with Neither Half Nor Full Bath:',\
      sum((test.HalfBath==0) & (train.FullBath==0)))

**************************************************
For Training Data Set:
**************************************************
Num of houses with Zero Bedroom: 6
Num of houses with Zero Garage Parking: 81
Num of houses with Zero Full Bath: 9
Num of houses with Zero Half Bath: 913
Num of houses with Neither Half Nor Full Bath: 3
**************************************************
For Test Data Set:
**************************************************
Num of houses with Zero Bedroom: 2
Num of houses with Zero Garage Parking: 76
Num of houses with Zero Full Bath: 3
Num of houses with Zero Half Bath: 921
Num of houses with Neither Half Nor Full Bath: 6


In [81]:
# Compute Total Bathrooms in a house
# and set those with zero bathroom with median bathroom number

train['TotBath'] = train.FullBath + 0.5*train.HalfBath;
test['TotBath'] = test.FullBath + 0.5*test.HalfBath;
train_test['TotBath'] = train_test.FullBath + 0.5*train_test.HalfBath;

train['TotBath'].replace(0, train['TotBath'].median(), inplace=True);
test['TotBath'].replace(0, test['TotBath'].median(), inplace=True);
train_test['TotBath'].replace(0, train_test['TotBath'].median(), inplace=True);

In [82]:
# Set those with zero bedroom with median bedfroom number 

train['BedroomAbvGr'].replace(0, train['BedroomAbvGr'].median(), inplace=True);
test['BedroomAbvGr'].replace(0, test['BedroomAbvGr'].median(), inplace=True);
train_test['BedroomAbvGr'].replace(0, train_test['BedroomAbvGr'].median(), inplace=True);

In [83]:
# Cook-up Feature:
# Bath_Capacity = TotBath / BedroomAbvGr

train['Bath_Capacity'] = train.TotBath / train.BedroomAbvGr
test['Bath_Capacity'] = train.TotBath / train.BedroomAbvGr
train_test['Bath_Capacity'] = train.TotBath / train.BedroomAbvGr

In [84]:
# Cook-up Feature:
# Parking_Capacity = TotBath / BedroomAbvGr

train['Parking_Capacity'] = train.GarageCars / train.BedroomAbvGr
test['Parking_Capacity'] = train.GarageCars / train.BedroomAbvGr
train_test['Parking_Capacity'] = train.GarageCars / train.BedroomAbvGr